Chapter 2.a. Core Expression Types
=======

Expressions in **Prove-It** are fundamental and versatile.  In order to prove some statement to be true, you must be able to express the statement that you want to prove, and express each axiom and each theorem that is used to construct the proof as well as intermediate judgments along the way.  
**Expressions** form the basis for **judgments** that are used in the derivation steps of a **proof** and will be discussed later.
An expression is a tree-like data structure (technically a directed acyclic graph, DAG, since there may be multiple instances of the same sub-expression).  As a Python object, its base class is `proveit._core_.expression.expr.Expression` (also aliased as `proveit.Expression` via "`from ... import ...`" Python statements within `__init__.py` files):

In [ ]:
from proveit import Expression
%begin basic_expressions
Expression

Each `Expression` object has a class that is derived one of the "primitive" `Expression` classes listed below (each of which is derived from `proveit._core_.expression.expr.Expression`).  These "primitive" types of expressions, in the order that we discuss them below, are:
* <a href="#Variable">`Variable`</a>: A label that is interchangeable (as long as it is kept distinct from different labels) with no intrinsic meaning.  It is often represented by a single letter ($a$, $b$, $x$, $y$, etc.) but can have any representation.
* <a href="#Literal">`Literal`</a>: A label that is not interchangeable and has an intrinsic meaning.  Specific operators ($\lnot, \land, +, \times$, etc.) and specific irreducible values ($\top, \bot, 0, 5$, etc.) are all `Literal`s.  Furthermore, a problem-story `Variable` in a particular theory package, representing some unknown but particular value, should also be a `Literal` (e.g., "Ann has $a$ apples...").
* <a href="#ExprTuple">`ExprTuple`</a>: A list of `Expression`s that is used, for example, when there are multiple *operators* or *operands* of an `Operation`, or multiple *parameters* of a `Lambda`.
* <a href="#Operation">`Operation`</a>: The application of *operator(s)* on *operand(s)*.  For example, $0 + 5 + 8$ and $1 < a \leq b < 3$ are examples of *operation* expressions.  The **Prove-It** library defines many *types* derived from the *operation* type (e.g., for each specific operation), but all *operations* have the same behavior with respect to reduction rules that will be described in a later tutorial.
* <a href="#Conditional">`Conditional`</a>: An expression that has one *condition* and one *value*.  The **Conditional** is reducible to its *value* if and only if the *value* is true.  For example $\{P(x) \textrm{ if } Q(x).$ reduces to $P(x)$ if $Q(x)$ is true.  Otherwise, the **Conditional** is not defined (i.e., it is not reducible).
* <a href="#Lambda">`Lambda`</a>: A mapping defined by *parameter* `Variable`s (and/or *iterations* of *indexed variables* described below with the `Iter` and `IndexedVar` core classes) transforming to some *body* `Expression`.  For example, $(x, y, z) \mapsto \{x+y/z~|~(x \in \mathbb{R}) \land (y \in \mathbb{R}) \land (z \in \mathbb{R}) \land (z \neq 0)\}$ is a `Lambda` with a `Conditional` *body* that converts three real numbers $x, y, z$ to $x + y/z$ as long as $z$ is not zero.  Note that a `Lambda` introduces `Variables` into a new scope via the *parameters*.  These *parameter* `Variable`s are said to be *bound* in this new scope; occurrences outside this scope are not deemed to be the same thing. There is a special category of `Operation` types called `OperationOverInstances` that are essentially functionals acting on a `Lambda` operand.  $\forall$, $\exists$, $\sum$, and $\prod$ are examples of `OperationOverInstances`.  For example, $\forall_{x~|~Q(x)} P(x)$ is an expression that translates to "$P(x)$ is true for all values of $x$ for which $Q(x)$ is also true".  Internally, this is represented as an $\forall$ operator acting on the `Lambda` map with a `Conditional` *body*: $x \mapsto \{P(x)~|~Q(x)\}$.
* <a href="#NamedExprs">`NamedExprs`</a>: A mapping from keyword strings to `Expression`s.  This can be used to prevent ambiguity of an expression's internal representation.
* <a href="#ExprRange">`ExprRange`</a>: Represents a range of expressions with a *parameter* going from a *start* index to an *end* index in successive unit increments ($+1$).  It contains a `Lambda` to define each expression in the range as a function of the *parameter* value (the index).
For example, $1/(x+i) + ... + 1/(x+j)$ is essentially an `Operation` with `+` as the *operator* and an `ExprRange` as the *operand*.  This `ExprRange` in this case is $1/(x+i), ..., 1/(x+j)$.  The sub-expression of this `ExprRange` may be denoted as $k \mapsto \{1/(x+k)~|~k \in i .. j\}$ where $k$ is an arbitrary `Lambda` *parameter* and $i .. j$ denotes an integer interval. 
* <a href="#IndexedVar">`IndexedVar`</a>: A special kind of `Operation` that indexes a `Variable`.  It treats the `Variable` being indexed as the *operator* and the *index* as the *operand*.  For example: $a_1$, $x_i$, $x_{i+1}$, etc.  An *iteration* if *indexed* *variables* acts as a collection of labels, however.  For example: $a_1, ..., a_n$ is a collection of $n$ labels and can be used as *parameters* of a `Lambda` map and has special substitution rules in which an `Iter` may be expanded to multiple entries within an `ExprTuple`.

These "primitive" types are special for the following reason.  In order to automatically or manually verify the correctness of a proof in **Prove-It**, one is only required to know the primitive base class of the expressions in the expression DAGs involved in the proof.  The way that the expression is to be formatted may be defined by the derived class, but this is a dressing which is not actually verified by **Prove-It** in any way.  For this reason, in order to really understand the axioms that were used in a proof, or understand what was actually proven, one must view the full expression DAG and understand its structure with respect to the primitive types.

Other classes are derived from these primitives for the purpose of defining specific ways of formatting the expression (as a string or as $\LaTeX$) and to provide methods for convenience and automation for manipulating expressions of that specific type.  The most common "primitive" type used as a base class is the `Operation`; there are a wide variety of kinds of operations (logical operations such as $\land$, $\lor$, $\lnot$, etc.; the $\forall$ and $\exists$ quantifiers, number operations such as $+$, $-$, etc.).  Each one of these has its own methods that are convenient for applying axioms and theorems specific to the operation.

A few non-privitive expression casses are define in the core (`proveit._core_`) for organizational purposes.  These are:
* <a href="#Function">`Function`</a>: An `Operation` that uses function-style formatting by default.  For example, $f(x)$, $P(x, y)$, etc.  Beyond formatting, these are no different than `Operation` types.
* <a href="#OperationOverInstances">`OperationOverInstances`</a>: An `Operation` whose operand is a `Lambda` expression.  Often, these `Lambda` expression has a `Conditional` body.  The idea is that it operates over the domain of instances of the `Lambda` parameter variables for the the *condition* of the `Conditional` is satisfied.  Quantifiers are `OperationOverInstances`.  $\forall_{x, y~|~Q(x, y)} P(x, y)$ is internally represented by Prove-It as an operation of the $\forall$ operator acting on $(x, y) \mapsto \left\{P(x, y) \textrm{ if } Q(x, y)\right.$.  $\sum_{k = 1}^{N} f(x)$ is internally represented by Prove-It as an operation of the $\sum$ operator acting on $k \mapsto \left\{f(x) \textrm{ if } k \in 1 .. N\right.$.
* <a href="#ConditionalSet">`ConditionalSet`</a>: An expression that has one or more *conditions* with associated *values*.  For example, consider this definition for the absolute value of an integer: $\{x~\textrm{ if }~x \geq 0; -x~\textrm{ if }~x < 0.$ A *value* is only pertinent if its associated *condition* is satisfied (a **judgment**).  Via axioms, a `Conditional` equates to one of its *values* if its associated *condition* is proven the be true and all of its other *conditions* are proven to false.  With respect to substitution rules, the substitution of a *value* may utilize the fact that its *condition* must be satisfied in order to be pertinent, and therefore its *condition* may be assumed within the context of performing substution for the *value*.

Developers and users may derive new classes from core `Expression` classes (`Literal` and `Operation` classes in order to make their own `Expression` classes that have special formatting rules for displaying the **expression** and methods for manipulating and utilizing the expression (i.e., applying theorems or axioms as will be discussed in later tutorial chapters).  Ideally, the formatting should be in direct correspondence with the structure of the `Expression` as a true reflection of the internal representation, but there is no enforcement mechanism to ensure that this is the case.  Users are advised to inspect the fully explicit expression DAGs of important axioms and theorems (or rely on crowdsource checking).  Conveniently, clicking on any **expression** rendered as LaTeX will link to a notebook that, when executed, will reveal this internal structure.  (Alternatively, the `exprInfo()` method may be called).

The remainder of this tutorial chapter will show examples and discuss details of the different core `Expression` classes and discuss their distinct behaviors with respect to *substitution* as well as *meaning* versus *style* equivalence.

The following are related but distinct manipulations in **Prove-It**:
* *Expression substutition*: creating a new **expression** from an existing **expression** by swapping one or more sub-expression(s) for other sub-expression(s) by using the `substituted` method.  This creates a new `Expression` but has nothing to do, intrinsically, with proving statements.
* *Instantiation* derivation step: deriving a new **judgment** from an existing **judgment** via *expression substitution* using specific rules and limitations to ensure that the derivation is sound.  This will be discussed in a later tutorial chapter.
* *Equality substitution*: using the `substitution` axiom of `proveit.logic.equality` to prove that $f(x) = f(y)$ given $x = y$.  This uses *instantiation* of the `substitution` axiom which states $\forall_{f, x, y~|~x=y}~(f(x) = f(y))$.  This specifically involves statements (**judgments**) with the `proveit.logic.equality.equals._operator_` **Literal** (the `=` sign). This will be discussed in a later tutorial chapter.

The focus here will be *expression substitution* with some mention of the additional restrictions applicable to the *instantiation* derivation step before delving into more details in later chapters.  It is important to understand the distinctions among these types of manipulations.

<a name="#Label"></a>Labels (Variables, IndexedVars, and Literals)
================================

In [ ]:
from proveit import Label

The `Variable` and `Literal` classes both derive from `Label`.  A `Label` is created with a string and a LaTeX format specification to determine how it is displayed.  In a Jupyter notebook, the LaTeX is rendered and presented in the output:

In [ ]:
tri = Label(stringFormat='triangle', latexFormat=r'\triangle')

After importing anything from `proveit` in a Jupyter notebook, whenever assignments are made at the end of an input cell the output shows the name of the assigned Python variable (not to be confused with a **Prove-It** *Variable*) followed by ':' and its rendered value.  In the above input, we have assigned `tri` to a `Label` object we have created which renders as a triangle shape.  For convenience, this Python variable name and shape appear in the output without any additional code.  The string format is presented when the object is converted to a string and is a useful alternate representation (used, for example, in error messages that are displayed as strings).

In [ ]:
str(tri)

The breakdown of an **expression** into its DAG structure is displayed by calling `exprInfo()`.  Clicking on the **expression** is also a way to view this information, linking to a page that shows a canonical way to build the **expression** and calling `exprInfo()` on that built **expression**.  Here we see that the `tri` is a `Label` with no sub-expressions.

In [ ]:
tri.exprInfo() # the expressions and sub-expressions are numbered with the top level being zero.

`Label` was not one of the core **expression** types listed above because it is not intended to be used except as the base class of `Variable`, `IndexedVar`, and `Literal`.  Notwithstanding, `Label` is displayed as the core type.
Two Label's are regarded to be the same expression if and only if the class and both formats (string and LaTeX), are the same:

In [ ]:
assert tri == Label('triangle', r'\triangle') # equal b/c class (Label) is the same and
                                              # the string and LaTex formats are the same
    
assert tri != Label('tri', r'\triangle')      # not equal when class or either format is different
                                              # Class for both is Label, but string format different

Let's make another label to test out `substituted()` method.

In [ ]:
sq =  Label(stringFormat='square', latexFormat=r'\Box')

Using *expression substitution*, we can change one `Label` into another or into any other `Expression`.  Later we'll see that *instantiation* is more restrictive, only being applicable to `Variable`s and not any `Label`, but for *expression substitution* this is fine.

In [ ]:
tri.replaced({tri:sq})

Notice that the `replaced()` method is non-mutating, leaving the tri `Label` as it was originally defined:

In [ ]:
tri

<a href="#Variable"></a>Variable
===========
A `Variable` is an interchangeable <a href="#Label">Label</a> with no formal, contextual meaning.

In [ ]:
from proveit import Variable
x = Variable('x')  # the string and latex formats
                   # are the same by default

In [ ]:
# different string and latex formats:
omega = Variable(stringFormat='omega', latexFormat=r'\omega') 

Convenient `Variable`s and other `Expression`s are accessible from `_common_.py` modules (generated from `_common_.ipynb` notebooks).  That is, `_common_.py` modules hold common expressions.  For example, from <a class="ProveItLink" href="../packages/proveit/_common_.ipynb">proveit.\_common\_</a> we can import a collection of commonly-used variables:

In [ ]:
from proveit._common_ import a, b, c, x, y, z, alpha
from proveit import ExprTuple
ExprTuple(a, b, c, x, y, z, alpha)

We used an <a href="#ExprTuple">ExprTuple</a> to conveniently render all of those `Variable`s as a single `Expression`.

Expression information for a `Variable` is similar to that for a `Label`, but with the different core type and class:

In [ ]:
alpha.exprInfo()

Just as we saw for the generic `Label` `Expression`s, we can use *expression substitution* to exchange one `Variable` for another or for any other `Expression`.  Furthermore, we will see in a later tutorial chapter that `Variable`s have special properties with respect to *instantiation* because they are defined as interchangeable labels.

In [ ]:
alpha.replaced({alpha:sq}) # we can substitute a Variable for any other Expression

<a name="#Literal"></a>Literal
-------

A `Literal` is another kind of <a href="#Label">Label</a>.  In contrast to `Variable`s, `Literal`s have a formal, contextual meaning.

In [ ]:
from proveit import Literal

Below are different scenarios in which a `Literal` could appropriately be used.

### Specific operators

In [ ]:
TIMES = Literal(stringFormat='*', latexFormat=r'\times')

In [ ]:
FACTORIAL = Literal('!')

In [ ]:
SUMMATION = Literal('sum', r'\sum')

We will see in the <a href="#Operation">Operation</a> section that the *operator* of an `Operation`-derived class should be stored as a Python class variable called `_operator_`.  We are using module-level Python variable names just for these examples.

### Constant values

In [ ]:
FIVE = Literal('5') # Literal constants are also ALL-CAPS

In [ ]:
TRUE = Literal('true', r'\top')

In [ ]:
FALSE = Literal('false', r'\bot')

Such constants will typically be defined as **common expressions** in the appropriate **context**, to be discussed in a later tutorial.

### Contextual "variables"

This is appropriate when variables are given a specific meaning within the context of a problem.  For example, consider a math "story problem" where Andrea has $a$ apples and Bill has $b$ bananas.  These are variables in a sense, but here they have contextual meaning so they should be `Literal`s.  However, suppose one wants to prove a general theorem for any number of Andrea's apples and Bill's bananas.  One may start by using the $a$ and $b$ `Literal`s for convenience for some contextual "lemmas" (as an aside, though, **Prove-It** does not distinguish between lemmas and theorems), but then demote them to `Variable`s in order to make a statement of the form $\forall_{a, b} P(a, b)$ (`Literal`s may not be quantified over).  This is done by using a process called axiom elimination that will be discussed later.  For now, we just note that "variables" with contextual meaning need to be `Literal`s.

In [ ]:
aLit = Literal('a')

In [ ]:
bLit = Literal('b')

In [ ]:
# different because one is a Variable and one is a Literal
assert a != aLit 
# same after converting the Literal to a Variable
assert a == aLit.asVariable() 

### Literal substitution
It is possible to perform *expression substitution* on a `Literal`.

In [ ]:
aLit.replaced({aLit:bLit})

In [ ]:
aLit.replaced({aLit:omega})

As noted earlier, the `substituted()` method is non-mutating, leaving the aLit `Literal` as it was originally defined:

In [ ]:
aLit

While we are able to replace a `Literal` via *expression substitution*, we may not replace them via *instantiation* because `Literal`s are not interchangeable labels.

### Literal contexts

A `Literal` is distinguished not only by its formatting but also by its **context**.  Axioms and theorems of **Prove-It** are organized via **contexts** which we will discuss later in more detail.  Each `Literal` belongs to a **context** which is typically the one corresponding to the package (directory) in which it is defined.  This is particularly important for *contextual variable* types of `Literal`s that are very *context*-specific.

In [ ]:
from proveit import logic

In [ ]:
logic.TRUE

*If the above output is not the $\top$ symbol when you execute this, you probably need to <a class="ProveItLink" href="tutorial00_introduction.ipynb#build">build or download the Prove-It database</a>.*

In [ ]:
# When the context is different, they are not the same.
TRUE == logic.TRUE

The detailed expression information reveals the differences:

In [ ]:
TRUE.exprInfo(details=True)

In [ ]:
logic.TRUE.exprInfo(details=True)

**Expressions** are also distinguished by their class (derived from the *core type*).  In the above example, `logic.TRUE` is an object of the `proveit.logic.boolean.booleans.TrueLiteral` class.  `logic.TRUE` and `logic.FALSE` have an `evalEquality(..)` method defined in their respective classes which is a convenient method for deriving any of the following: $(\top = \top) = \top$, $(\bot = \bot) = \top$, $(\top = \bot) = \bot$, or $(\bot = \top) = \bot$ (these are proven theorems within `proveit.logic`). For example:

In [ ]:
logic.TRUE.evalEquality(logic.FALSE)

That is a sneak preview of a judgment (**Judgment** object) that uses the turnstile notation, $\boldsymbol{\vdash}$, to indicate that the expression is a proven statement.  There will be more on this in later tutorial chapters.

<a name="ExprTuple"></a>ExprTuple
=========

An `ExprTuple` is an ordered collection of **expression** objects which we previewed briefly already.  Here is another example:

In [ ]:
et = ExprTuple(a, b, c, TRUE, FIVE, a, alpha)

An `ExprTuple` may be used as the *operands* or *operations* of an `Operation`, the *parameters* of a `Lambda` map, or the *values* and *conditions* of a `Conditional`.  A Python tuple (or any iterable) will automatically be converted to an `ExprTuple` when it is passed in for any of these roles when constructing an `Operation`, `Lambda`, or `Conditional`. 

Substitution typically carries forward in a straightforward manner.

In [ ]:
et.replaced({a:FALSE})

Note that the *substitution* occurs on all relevant instances.  And, as always, `substituted` is non-mutating:

In [ ]:
et

The **expression** constituents of an `ExprTuple` are known as *entries*:

In [ ]:
et.entries[5]

The enumerated mathematical objects represented by the `ExprTuple` are known as *elements*.  The *elements* and *entries* are in correspondence except when the `ExprTuple` contains an `Iter`.  A contained `Iter` is a single *entry* but may represent multiple elements.  For example, the second *entry* of $(a, b_1, ..., b_n)$ is the `Iter` $b_1, ..., b_n$, but the second *element*, assuming $n \geq 1$, is $b_1$.  We will revisit this in the <a href="#Iter">Iter</a> section.

`ExprTuple` *substitution* is more interesting when an `Iter` containing an `IndexedVar` is expanded such that the `Iter` splits into multiple *entries*.  This will be demonstrated in the <a href="#IndexedVar">IndexedVar</a> section.  

<a name="Operation"></a>Operation
=========

This type of **expression** represents an applied operation.  It contains *operator(s)* and *operand(s)*.  The default formatting of an `Operation` with one *operator* and more than one *operand* is to place the *operator* between each successive pair of *operands* as follows.

In [ ]:
from proveit import Operation
triple_prod = Operation(TIMES, [x, y, omega])

A `Function` is derived from `Operation`.  It behaves in the same manner except it formats it in a "function" style:

In [ ]:
from proveit import Function

# Simple case: a Variable operator with a single Variable operand
f = Variable('f')
fx = Function(f, x)

Common forms of these are available from <a class="ProveItLink" href="../packages/proveit/_common_.ipynb">proveit._common_</a>:

In [ ]:
from proveit._common_ import fx, gx, Px, Qx
ExprTuple(fx, gx, Px, Qx)

An `Operation` may have a single *operator* that is a *label* (*variable* or *literal*), or it may have multiple *operators* that are *labels* and/or *iterations of indexed variables.*

In [ ]:
try:
    Operation(fx, fx)
    assert False, "Expecting a TypeError; should not make it to this point"
except TypeError as e:
    print("Expected TypeError: %s"%str(e))

The `operators` and `operands` attributes access the *operators* and *operands* respectively as `ExprTuple`s:

In [ ]:
triple_prod.operators

In [ ]:
triple_prod.operands

If there is only one *operator* or *operand*, it may be accessed via the respective `operator` and `operand` attributes: 

In [ ]:
triple_prod.operator

In [ ]:
fx.operand

In [ ]:
# An Operation does not have an `operand` attribute
# if there is more than one *operand*.
hasattr(triple_prod, 'operand')

Looking at the expression information, we see that `triple_prod` has `operands` that is an `ExprTuple` but a single `operator` that is a `Literal`.

In [ ]:
triple_prod.exprInfo()

### Defining an `Operation` sub-class

Let's derive a couple of classes from `Operation` to represent operations with specific literal operators.
This is commonly done within Prove-It library modules.  We use the `%load` magic command to display (and execute) the contents of <a href="demo_operations.py">demo_operations.py</a>.

In [ ]:
# %load demo_operations.py
'''
Module that defines a Operations for demonstration
purposes in this tutorial.
'''

from proveit import Operation, Literal

class Factorial(Operation):
    # _operator_ is a special class variable name, defining specific literal operator of the Operation class.
    # It is not only used for default formatting but also when performing substitutions for rebuilding expressions.
    _operator_ = Literal('!')
    
    def __init__(self, operand):
        # creates the Operation with FACTORIAL as the operator and the provided operand as its only operand.
        Operation.__init__(self, Factorial._operator_, operand) # initializes self.operand

    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.string(fence=True) + Factorial._operator_.string()
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.latex(fence=True) + Factorial._operator_.latex() 

class Multiply(Operation):
    
    # This operator Literal has a LaTeX format that differs from the string format.
    _operator_ = Literal('*', r'\times')
    
    def __init__(self, *operands): # takes a list of arguments as the operands
        # creates the AssociativeOperation with TIMES as the operator and any number of operands.
        Operation.__init__(self, Multiply._operator_, operands)
    
    # The default formatting will display the operator between the operands


In order to work properly, however, we need to import these classes from the module (as Prove-It will use the `__file__` attribute of this module for its internal purposes).

In [ ]:
from demo_operations import Factorial, Multiply

A `Factorial` is an `Operation` with "!" as the format of the `Literal` operator.  Its formatting is altered from the default but is still true to the core `Expression` structure (the formatting is a reflection of the internal structure).

In [ ]:
# Now we can make an object with this new class
x_factorial = Factorial(x)

In [ ]:
# show core structure
x_factorial.exprInfo()

In the Jupyter notebook setting, we typically only use LaTeX formatting, but it is important to define the string formatting as well for whenever it is needed (e.g., when error messages are displayed).  The `fence=True` above indicates that parentheses should be used when it could be ambiguous otherwise.  Our example above has no ambiguity, so parentheses are not used.  We will show a case below where the parentheses are required.

Next we consider a multiple *operand* example using the `Multiply` class that we imported from <a href="demo_operations.py">demo_operations.py</a> above.

In [ ]:
# Demonstrating an Operation with multiple operands
mult_expr = Multiply(x, FIVE, omega)

In [ ]:
# Let's nest Operations and show proper fencing behavior
nested_operation = Factorial(mult_expr)

In [ ]:
nested_operation.exprInfo()

### Operand substitution

*Substitution* of *operands* is straightforward in the way it works.

In [ ]:
x_factorial.replaced({x:omega})

Notice that the substituted() method is non-mutating, leaving the **xFactorial** expression as it was originally defined:

In [ ]:
x_factorial

We can also substitute an entire expression in for the variable $x$:

In [ ]:
nested_operation_from_sub = x_factorial.replaced({x:mult_expr})

Internally, however, there is specific machinery required to regenerate these objects and construct them with the proper classes.  In order for this machinery to work, the `_operator_` class attribute must be properly defined (e.g., `Factorial._operator_` and `Multiply._operator_`).  To demonstrate that this is working properly, note that the classes displayed in the detailed expression info from `exprInfo()` are as they should be:

In [ ]:
nested_operation_from_sub.exprInfo(details=True)

Furthermore, this expression that we obtained via substitution is equivalent to the one constructed directly:

In [ ]:
print("Expressions, generated in different ways, are the same:", 
      (nested_operation_from_sub == nested_operation))

### Operator substitution

The *operator* itself may likewise be *substituted*, but with some interesting extra capabilities.

It is straightforward to substitute a **Variable** *operator* with another **Variable** (i.e., relabel). Here we relabel the $f$ in $f(x)$ to $g$ to obtain $g(x)$:

In [ ]:
from proveit._common_ import g
fx.replaced({f:g}) # Variable operator to a different Variable

And it is straightforward to *substitute* a **Variable** *operator* with a **Literal**.

In [ ]:
x_factorial_from_fx = fx.replaced({f:FACTORIAL})

In [ ]:
x_factorial_from_fx.exprInfo(details=True)

In [ ]:
assert x_factorial_from_fx == x_factorial # Same

It is able to generate the new **expression** in the appropriate `Factorial` class via the internal mechanisms that use its `_operator_` class attribute.

Performing an *expression substitution* of a `Literal` *operator* within an `Operation` class will typically be blocked because it will not know how to make the new **expression**:

In [ ]:
from proveit import OperationError
# But this will typically be prevented (appropriately) in trying to remake a derived Operation class:
try:
    x_factorial.replaced({FACTORIAL:SUMMATION})
    assert False, "Expecting an OperationError error; should not make it to this point"
except OperationError as e:
    print("EXPECTED ERROR:", e)

### <a name="OperationSub"></a>Operation substitution

The more interesting and useful case is to *substitute* the `Operation` itself. Here we convert $f(x)$ to $x!$ by substituting a `Lambda` mapping for the function variabe $f$:

In [ ]:
from proveit import Lambda
fancy_factorial_map = Lambda(y, Multiply(y, Factorial(y)))

The <a href="#Lambda">`Lambda`</a> core type will be discussed in more detail below.  We use it here for a quick example.

In [ ]:
fx.replaced({f:fancy_factorial_map})

Note that the operand is still $x$ (not $y$).  This is substituting the *operation*, not the operand.  If desired, however, it could be substituted simultaneously:

In [ ]:
fancy_omega_fn = fx.replaced({f:fancy_factorial_map, x:omega})

In [ ]:
fancy_omega_fn.exprInfo(details=True)

Note that by substituting $f$ with a `Lambda` expression, the entire `Operation` is substituted, not just the *operator*.  This is why `Lambda` *operator*s are not allowed (only `Label` operators); otherwise, such a substitution could be ambiguous or confusing (should it substitute the `Operation` or substitute the *operator* with the `Lambda` expression?).  This also highlights the fact that **Prove-It** is not a functional programming or lambda calculus.  Rather, **Prove-It** is designed for manipulating expressions as desired, and *operation substitution* is a useful and powerful, yet relatively straightforward tool for doing so.

<a name="Conditional"></a>Conditional
====

A `Conditional` has one or more *values* associated with one or more *conditions*.  A given *value* is only pertinent if its associated *condition* is satisfied (known to be true).  MUST UPDATE THIS SECTION.  A CONDITIONAL ONLY HAS ONE VALUE AND CONDITION.  A CONDITIONAL SET

In [ ]:
from proveit import Conditional, ConditionalSet

To create the `Conditional`, supply the *values* and *conditions* as separate arguments in the constructor.  This example invokes the `Neg`, `Less`, and `GreaterEq`  <a href="#Operation">`Operation`</a>s as well as the `zero` <a href="#Literal">`Literal`</a>)

In [ ]:
from proveit.number import zero, Neg, Less, GreaterEq
abs_cond = Conditional((x, Neg(x)), 
                       (GreaterEq(x, zero), Less(x, zero)))

In [ ]:
# The string formatting is somewhat different:
str(abs_cond)

The `conditions` and `values` attributes may be used to access the *conditions* and *values* respectively.

In [ ]:
abs_cond.condition

In [ ]:
abs_cond.value

This is the expression information breakdown for our `Conditional` example.

In [ ]:
abs_cond.exprInfo()

It is actually common to have only one *condition*, indicating a value that is only defined when the condition is satisfied.

In [ ]:
from proveit.logic import Equals, NotEquals
from proveit.number import zero, Mult, frac
div_cancel_cond = Conditional(Equals(Mult(x, frac(y, x)), y),
                              NotEquals(y, zero))

It will then have singular `condition` and `value` attributes.

In [ ]:
div_cancel_cond.condition

In [ ]:
div_cancel_cond.value

<a name="Lambda"></a>Lambda
====

This type of **expression** represents a mathematical mapping or function.  It contains *parameter(s)* (one or more *variables* and/or *iteration* of *indexed* *variables* that will be discussed in the <a href="#IndexedVar">IndexedVar</a> section below) and a *body* (any **expression** that the *parameters* are to be mapped into).

In [ ]:
from proveit import Lambda

In [ ]:
# maps any value to 5
map_to_5 = Lambda(x, FIVE)

In [ ]:
# Let's look at the Expression info
map_to_5.exprInfo()

Here is an example with multiple *parameters* represented by an `ExprTuple` and a `Conditional` *body* .  This maps a pair of real numbers to the first element (invoking the `And`, `Add`, and `InSet` <a href="#Operation">`Operation`</a>s and using the `Reals` <a href="#Literal">`Literal`</a>):

In [ ]:
# Can have multiple arguments
from proveit.logic import InSet, And
from proveit.number import Add, Reals
from proveit import Conditional
lambda_expr = Lambda([x, omega], 
                     Conditional(Add(x, omega), 
                                 And(InSet(x, Reals), 
                                     InSet(omega, Reals))))

In [ ]:
lambda_expr.exprInfo()

The `parameters` and `body` attributes access these respective sub-expressions:

In [ ]:
lambda_expr.parameters

In [ ]:
lambda_expr.body

If a `Lambda` map has one *parameter*, it will have a `parameter` attribute.

In [ ]:
single_param_lambda = Lambda(x, Mult(FIVE, a))

In [ ]:
single_param_lambda.parameter

### Lambda relabeling and relabeled equivalence

The choice of *parameter* *labels* has no effect on the meaning of a `Lambda` map.  We are free to "relabel" the *parameters* (via `Lambda.relabeled`) as long as it is done consistently and distinct labels remain distinct.

In [ ]:
lambda_expr

In [ ]:
lambda_expr_v2 = lambda_expr.relabeled({x:y})

We can even simultaneously swap labels:

In [ ]:
lambda_expr

In [ ]:
lambda_expr.replaced({x:omega, omega:x}, allow_relabeling=True)

In [ ]:
lambda_expr_v3 = lambda_expr.relabeled({x:omega, omega:x})

But we cannot allow a collision of the label mappings:

In [ ]:
from proveit import DisallowedParameterRelabeling
try:
    lambda_expr.relabeled({x:y, omega:y})
    assert False, ("Expecting a DisallowedParameterRelabeling error; "
                   "should not make it to this point.")
except DisallowedParameterRelabeling as e:
    print("EXPECTED ERROR: ", e)

In [ ]:
try:
    lambda_expr.relabeled({x:omega})
    assert False, ("Expecting a DisallowedParameterRelabeling error; "
                   "should not make it to this point.")
except DisallowedParameterRelabeling as e:
    print("EXPECTED ERROR: ", e)

Also, an exception is raised if we attempt to relabel to a non-Variable.

In [ ]:
try:
    lambda_expr.relabeled({x:fx})
    assert False, ("Expecting a TypeError error; "
                   "should not make it to this point.")
except TypeError as e:
    print("EXPECTED ERROR: ", e)

Moreover, **Prove-It** recognizes the different valid relabeled versions of *Lambda* expressions as the *same* **expression** with respect to *meaning*.  They are regarded as equal.

In [ ]:
assert lambda_expr == lambda_expr_v2 == lambda_expr_v3

These are recognized as having distinct *styles* of presentation, however.    **Prove-It** provides flexibility with respect to presentation style of expressions while recognizing the *meaning* equivalence when style is the only distinction.  This is the first of many examples to be encountered.

### Lambda scope and substitution rules

It is important to understand that a `Lambda` expression defines a new scope within its *body* with respect to the *parameters*.  This has two important consequences with respect to substitution rules.

The first consequence of the internal `Lambda` scope is that the `Lambda` masks the substitution of variables that happen to be the same as any of the `Lambda` parameters.  For example,

In [ ]:
tuple_with_lambda = ExprTuple(x, lambda_expr)

In [ ]:
tuple_with_lambda.replaced({x:aLit}) 

Here we see that $x$ is altered external to the `Lambda` expression, but the $x$ within the `Lambda` expression is left alone.  This is fine because the internal $x$ is essentially a different $x$; the scope is different.

For convenience, `substituted` will perform relabeling if the replacement of a `Lambda` *parameter* is another `Variable`.  For example,

In [ ]:
tuple_with_lambda.replaced({x:y}) 

The inner and outer $y$ *variables* are still regarded as distinct, but it doesn't hurt to relabel the one within the `Lambda` as we wish.  If we do not want this behavior when performing *expression substitution* or *instantiation*, we can *relabel* the map with a distinct variable first.  This feature simply provides a convenient way to perform inner relabeling simultaneously with other substitutions.

The other consequence for of `Lambda` scoping with respect to substitution is that we must not allow any expression within the `Lambda` *body* to be replaced with something that contains one of its *parameters*.  That is because those *parameters* are, with respect to scope, distinct from anything outside of the `Lambda` expression.  So bringing in a replacement defined for external use would have a different meaning when applied internally, and this must not be allowed.  Otherwise, you could alter the meaning of the expression in ways that can invalidate a proof (e.g., via a *instantiation* step which relies upon *expression substitution* internally).  Thus, the parameters of the Lambda function are "reserved" and may not be used in substitution. 

Consider the mapping defined below:

In [ ]:
map_to_new_var = Lambda([x, omega], z)

Expressions involving $x$ and/or $w$ are not allowed to be used in the substitutions for $z$. For example, we are not allowed to transform this map to $(x, \omega) \mapsto \omega$ via substitution.  Prove-It will automatically avoid this via automated relabeling.

In [ ]:
map_to_new_var.replaced({z:omega})

And we are not allowed to change the map to $(x, \omega) \mapsto f(x)$ via substitution.  Again, automatic relabeling avoids this issue.

In [ ]:
map_to_new_var.replaced({z:fx})

If it does not involve any of the parameter variables, however, that is okay.

In [ ]:
tuple_with_lambda.replaced({x:y}) 

In [ ]:
map_to_new_var.replaced({z:FIVE})

### Applying the Lambda map

In addition to the `substituted` method, the `Lambda` class has an `apply` method.  As arguments, the `apply` method should be given the operands to be mapped.  The `apply` method will apply the proper substitution replacements to the `Lambda` *body* to effect the application of the map to the operands.  Internally, <a href="#OperationSub">*operation substitution*</a> is effected via the `Lambda.apply` method.

In [ ]:
lambda_expr

In [ ]:
lambda_expr.apply(fancy_omega_fn, y)

### Mocking up **Lambda** operators

What if we wanted to represent an expression involving a `Lambda` function applied to operand(s).  We cannot do this using the `Lambda` and `Operation` expression types directly because an `Operation` is only allowed to have *variable* or *iterated indexed variables* as *operators*.   But it would not be consistent with the **Prove-It** philosophy of *freedom of expression* to completely disallow representing such an expression if so desired.  We can, in fact, represent such a thing in **Prove-It** by making an `Operation` class whose *operands* are the `Lambda` function as well as the operand on which we wish to apply the function. As a demonstration, we define such an `Operation`, called `LambdaApplication`, in <a href="demo_lambda_app_operation.py">demo_lambda_app_operation.py</a>.

In [ ]:
# %load demo_lambda_app_operation
'''
Module that defines a LambdaApplication Operation
class for demonstration purposes in this tutorial.
'''

from proveit import Operation, Literal, NamedExprs

class LambdaApplication(Operation):
    _operator_ = Literal('LAMBDA_APPLICATION', r'{\rm LAMBDA\_APPLICATION}')
    
    def __init__(self, lambdaFn, operand):
        Operation.__init__(self, LambdaApplication._operator_, NamedExprs([('lambdaFn',lambdaFn), ('operand',operand)]))
        self.lambdaFn = self.operands['lambdaFn'] # The Lambda function operand
        self.lambdaOperand = self.operands['operand'] # The operand of the Lambda function

    @classmethod
    def extractInitArgValue(operationClass, argName, operator, operand):
        '''
        Given a name of one of the arguments of the __init__ method,
        return the corresponding value as determined by the operator and
        operand of the LambdaApplication Operation.
        (This is important so that Prove-It knows how to 'make' an altered
        copy of this Operation).
        '''
        assert isinstance(operand, NamedExprs), "Expecting LambdaApplication operand to be a NamedExprs object"
        if argName=='lambdaFn': 
            return operand['lambdaFn']
        elif argName=='operand':
            return operand['operand']

    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        return self.lambdaFn.string(fence=True) + '(' + self.lambdaOperand.string() + ')'
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        return self.lambdaFn.latex(fence=True) + '(' + self.lambdaOperand.latex() + ')'


In [ ]:
from demo_lambda_app_operation import LambdaApplication

In [ ]:
lambdaAppExpr = LambdaApplication(Lambda(y, Factorial(y)), x)

Note that the `_operator_` of the `LambdaApplication` is not displayed but is implicit in the context of the lambda application formatting.  Also, we made use of `NamedExprs` which will be discussed next.  Let us take a look at the expression information.

In [ ]:
lambdaAppExpr.exprInfo(details=True)

**Axioms** could then be created to define how the `LambdaApplication` **operation** actually operates, but **axioms** will be discussed in a later chapter.

NamedExprs
=========

`NamedExprs` is a composite **expression** that maps string keywords to sub-**expressions**.  The reason that this may be desired is to be explicit in the internal representation about the role of each sub-**expression** so there is no ambiguity between the internal and external representation.  A good example is the `lambdaAppExpr` $[y \mapsto y!](x)$ defined above, where one *operand* plays the role of the lambda function and the other *operand* plays the role of the lambda function's operand.

Here is a more rudimentary demonstration of using **NamedExpressions**:

In [ ]:
from proveit import NamedExprs
NamedExprs([('left',x), ('right',y), ('product',Multiply(x, y))])

To produce a `NamedExprs` object, you must supply a list of (keyword, `Expression`) pairs.  The keywords must be strings, but can be any string desired.  When the **NamedExprs** is displayed, it will show each of these pairs in the order that they were originally provided.

ExprRange
=========

It is common in mathematical notation to use ellipses ($...$) in representing an unspecified number of parameters or operands.  The `ExprRange` expression type serves this purpose.

To create an `ExprRange`, supply a *parameter*, *body*, and *start* and *end* indices.

In [ ]:
from proveit import ExprRange
from proveit._common_ import i, j, k
from proveit.number import one, Add, frac
it = ExprRange(k, frac(one, Add(x, k)), i, j)

For compactness, we only show the start and the end instances.  It is implicit that our index parameter ($k$, which is not explicitly shown) is that our index ranges from $i$ to $j$ in increments of $1$.  (Future style options could be implemented, if desired, to make the representation more explicit at the expense of being less compact without changing any of the inner workings of the `Iter` expression type.)

Looking at the expression information, we see that there are three sub-expression of the `Iter` object: the `lambda_map`, `start_index` and `end_index`.

In [ ]:
it.exprInfo()

The `lambda_map`, `start_index` and `end_index` attributes provides access to the sub-expression.  The `parameter` and `body` attributes provide convient access to some indirect sub-expressions (the sub-expressions of `lambda_map`).

In [ ]:
it.lambda_map

In [ ]:
it.parameter

In [ ]:
it.body

In [ ]:
it.start_index

In [ ]:
it.end_index

We have special formatting features when an `Iter` is used as *operands* of an `Operator`.  As stated previously, **Prove-It** is flexible in its formatting.

In [ ]:
add_it = Add(it)

To really understand how **Prove-It** recognizes an expression, one must look at the expression information. 

In [ ]:
add_it.exprInfo()

IndexedVar
=========

To truly understand what can be done with `Iter` expressions, we must see how they can be used with `IndexedVar` expressions.  On its own, an `IndexedVar` if fairly straightforward.  An `IndexedVar` is effectively an `Operation` where the *operator* is the variable being indexed and the *operand* is the *index*, but it does have some special properties related to <a href="#IterParams">iterated parameters</a> which justifies makin `IndexedVar` a core expression type.

In [ ]:
from proveit import IndexedVar
a_i = IndexedVar(a, i)

In [ ]:
a_i.exprInfo()

This "operator" must be a `Variable` or `IndexedVar`:

In [ ]:
try:
    IndexedVar(fx, i)
    assert False, ("Expecting a TypeError error; "
                   "should not make it to this point")
except TypeError as e:
    print("EXPECTED ERROR: ", e)

And `IndexedVar` may have multiple indices:

In [ ]:
a_ij = IndexedVar(a, (i, j))

The real purpose of the `IndexedVar` expression type is to be used within an iteration.  So there is a convenient method called `varRange` for generating a range of indexed variables.

In [ ]:
from proveit import varRange
a_i_to_j = varRange(a, i, j)

### <a name="IterParams"></a>Iterated parameters

When a `Lambda` expression has iterated parameters, a parameter entry that is an `varRange` over `IndexedVar`s, the `apply` method offers some interesting versatility.  Consider the following example representing a map which distributes a product over a sum of an unspecified number of terms.

In [ ]:
a_k = IndexedVar(a, k)
distribution_map = Lambda((a_i_to_j, b),
                          Add(ExprRange(k, Mult(a_k, b), i, j)))

We can apply a map which splits $a_i, ..., a_j$ into multiple entries.  In so doing, the `Iter` of the result is split in the same manner.  For example, consider the following substitution into $a_i, ..., a_j$:

In [ ]:
from proveit._common_ import m, n
x_1_to_m = varRange(x, one, m)
y_1_to_n = varRange(y, one, n)
a_sub = ExprTuple(x_1_to_m, y_1_to_n, z)

But to allow this as a subtitution, we must prove (or assume) that the lengths of these `ExprTuple`s match (that the have the same number of elements).  We could prove this assuming $j-i+1 = m+n+1$, but for this demonstration we will simply assume what we directly require to be true.

In [ ]:
from proveit.core_expr_types import Len
a_len = Len(ExprRange(k, k, i, j))
a_required_len = Equals(Len(a_sub), a_len)

Using `a_required_len` as an assumption, we can `apply`  `distribution_map` to the operands $(x_1, ..., x_m, y_1, ..., y_n, z)$ to obtain the following

In [ ]:
requirements = []
distribution_map.apply(*a_sub, b, assumptions=[a_required_len],
                      requirements=requirements)

The `requirements` is appended with facts that were needed to make this happen.  Here, there is just one requirement which is the statement we assumed.  It is a **Judgment** proven by *assumption*; these concepts will be discussed in future chapters.

In [ ]:
ExprTuple(*requirements)

Recall that the `apply` method is used in the implementation of <a href="#OperationSub">*operation substitution*</a>.  Iterated parameters can be used in that setting as well.  Furthermore, this code is used in *instantiation* of iterated parameters that are universally quantified which will be discussed in a later chapter.

We will remark here that there are limitations in this capability for the sake of simplicity.  We deal with multiple `IndexedVar`s in an iteration, but their subsitutions must be aligned.  For example,

In [ ]:
b_k = IndexedVar(b, k)
b_i_to_j = varRange(b, i, j)
dot_prod_map = Lambda((a_i_to_j, b_i_to_j),
                          Add(ExprRange(k, Mult(a_k, b_k), i, j)))

In [ ]:
requirements = []
dot_prod_map.apply(*a_sub, *a_sub, assumptions=[a_required_len],
                   requirements=requirements)

As long as the iteration ranges are aligned, we are okay

In [ ]:
x_1_to_n = varRange(x, one, n)
y_1_to_m = varRange(y, one, m)
b_sub = ExprTuple(y_1_to_m, x_1_to_n, z)

In [ ]:
b_required_len = Equals(Len(b_sub), a_len)

In [ ]:
requirements = []
dot_prod_map.apply(*a_sub, *b_sub, assumptions=[a_required_len,
                                               b_required_len],
                   requirements=requirements)

In [ ]:
ExprTuple(*requirements).withWrappingAt(1)

If the ranges are not aligned, this will fail

In [ ]:
b_sub = ExprTuple(y_1_to_n, x_1_to_m, z)

In [ ]:
b_required_len = Equals(Len(b_sub), a_len)

In [ ]:
from proveit import LambdaApplicationError
try:
    dot_prod_map.apply(*a_sub, *b_sub, assumptions=[a_required_len,
                                                   b_required_len],
                       requirements=requirements)
    assert False, "Expecting a LambdaApplicationError error; should not make it to this point"
except LambdaApplicationError as e:
    print("Expected LambdaApplicationError: %s"%str(e))

Such cases may still be handled, but the iterations must first be equated, via axioms/theorems, to split versions withcommon boundaries.  For example, given the following for the $a$ and $b$ substitutions

In [ ]:
a_sub

In [ ]:
b_sub

If we know that $m$ and $n$ are natural numbers and $m \leq n$, then we can split `a_sub` and `b_sub` respectively into

In [ ]:
from proveit.number import subtract
a_sub = ExprTuple(x_1_to_m, varRange(y, one, subtract(n, m)),
                  ExprRange(k, IndexedVar(y, Add(subtract(n, m), k)),
                            one, m), z)

In [ ]:
b_sub = ExprTuple(y_1_to_m, 
                  ExprRange(k, IndexedVar(y, Add(m, k)), 
                            one, subtract(n, m)),
                  x_1_to_m, z)

Now the `ExpRange` start and end indices line up: $1$ to $m$, $1$ to $n-m$, and $1$ to $m$, and the last entry is a singular element for both of them.  Now we can apply the map:

In [ ]:
a_required_len = Equals(Len(a_sub), a_len)
b_required_len = Equals(Len(b_sub), a_len)
dot_prod_map.apply(*a_sub, *b_sub, assumptions=[a_required_len,
                                               b_required_len],
                   requirements=requirements)

All of the variables that are indexed by the range parameter (which ranges from the start to the end) must be expanded together unless the indices of the expansion match with iteration indices, not just equal in length.

In [ ]:
b_k = IndexedVar(b, k)
b_i_to_j = varRange(b, i, j)
dot_prod_map = Lambda((a_i_to_j),
                      Add(ExprRange(k, Mult(a_k, b_k), i, j)))

In [ ]:
try:
    dot_prod_map.apply(*a_sub, assumptions=[a_required_len],
                       requirements=requirements)
    assert False, "Expecting a LambdaApplicationError error; should not make it to this point"
except LambdaApplicationError as e:
    print("Expected LambdaApplicationError: %s"%str(e))

Let's try an example where the indices do match.  This time, we'll do it properly by proving the equivalences via proofs.  (This is using some advanced concepts to be discussed in later chapters, but we'll probably rearrange this anyway).

In [ ]:
from proveit.number import subtract, Naturals
j_m_1 = subtract(j, one)
j_m_i = subtract(j, i)
j_m_i__in__Naturals = InSet(j_m_i, Naturals)
a_i_to_j__partition = \
    a_i_to_j.partition(j_m_1, [j_m_i__in__Naturals])

In [ ]:
a_i_to_j__partition.substitution(Lambda(x, Len(x)))

In [ ]:
i_to_j = ExprRange(k, k, i, j)

In [ ]:
i_to_j__partition = i_to_j.partition(j_m_1, [j_m_i__in__Naturals])

In [ ]:
i_to_j__partition.substitution(Lambda(x, Len(x)))

In [ ]:
a_sub = a_i_to_j__partition.rhs

Now, with sufficient assumptions, we can explicitly do the expansion for just the $a$ variable, but the $b$ variable will be forced to split at the same locations.

In [ ]:
a_required_len = Equals(Len(a_sub), a_len)
assumptions = [a_required_len, InSet(j, Naturals), j_m_i__in__Naturals]

In [ ]:
requirements = []
dot_prod_map.apply(*a_sub, assumptions=assumptions, requirements=requirements)

Note that we now have the extra requirement that the indices match, not just the lengths.

In [ ]:
ExprTuple(*requirements).withWrappingAt(1)

If the iteration parameter occurs anywhere other than as an index of an indexed variable, this will also spur the need for the requirement that the indices match and not just the lengths.  For example, consider

In [ ]:
dot_prod_and_more_map = Lambda((a_i_to_j, b_i_to_j),
                               Add(ExprRange(k, Mult(a_k, b_k, k),
                                             i, j)))

We cannot handle the following substitution because the indices do not match with the original iteration indices.

In [ ]:
a_sub, b_sub = (ExprTuple(x_1_to_m, y_1_to_n, z),
                ExprTuple(y_1_to_m, x_1_to_n, z))

In [ ]:
try:
    a_required_len = Equals(Len(a_sub), a_len)
    b_required_len = Equals(Len(b_sub), a_len)
    dot_prod_and_more_map.apply(*a_sub, *b_sub, 
                                assumptions=[a_required_len,
                                             b_required_len],
                                requirements=requirements)
    assert False, "Expecting a LambdaApplicationError error; should not make it to this point"
except LambdaApplicationError as e:
    print("Expected LambdaApplicationError: %s"%str(e))

But if we ensure that indices match up, we can do the expansion as we saw before.  Let's try this for a slightly modified map in which the Lambda parameters only covers the $a$ variables. 

In [ ]:
dot_prod_and_more_amap = Lambda((a_i_to_j),
                                Add(ExprRange(k, Mult(a_k, b_k, k), 
                                              i, j)))

In [ ]:
a_sub = a_i_to_j__partition.rhs

In [ ]:
a_required_len = Equals(Len(a_sub), a_len)
b_required_len = Equals(Len(b_sub), a_len)
assumptions = [a_required_len, b_required_len, InSet(j, Naturals), j_m_i__in__Naturals]

In [ ]:
requirements = []
dot_prod_and_more_amap.apply(*a_sub, assumptions=assumptions, requirements=requirements)

Again, we have the extra requirement that the indices match, not just the lengths.

In [ ]:
ExprTuple(*requirements).withWrappingAt(1)

In [ ]:
%end basic_expressions

# Next chapter: <a class="ProveItLink" href="tutorial02_proof_basics.ipynb">Proof Basics</a>

## <a class="ProveItLink" href="tutorial00_introduction.ipynb#contents">Table of Contents</a>